In [6]:
# bob.refresh {"cron": "15 2-12 * * *", "timeout": 40000}
# bob.static {"token": "2490a15907a84c6992746a301472670f93d69adfd300d878d6ca779bd1fa", "path": "/home/ftp/ftp/dev/wsr-public/data_output/WORLD_CONFIRMED.html", "filename": "../data_output/WORLD_CONFIRMED.html"}
# bob.static {"token": "a43c068d301b1bf55b676a083b4ee8fe8414f1802dfe0a2b70991b04a752", "path": "/home/ftp/ftp/dev/wsr-public/data_output/WORLD_ACTIVE_CASES.html", "filename": "../data_output/WORLD_ACTIVE_CASES.html"}
# bob.static {"token": "e69a785963750df22b746d3d35342a166a2832dc336e6e587049b0afe7f6", "path": "/home/ftp/ftp/dev/wsr-public/data_output/WORLD_DEATHS.html", "filename": "../data_output/WORLD_DEATHS.html"}
# bob.static {"token": "3bf81240bd15e2bb2560e1efdaeebd11252aead4a95563ab1d3a44d96505", "path": "/home/ftp/ftp/dev/wsr-public/data_output/WORLD_RECOVERED.html", "filename": "../data_output/WORLD_RECOVERED.html"}
# bob.static {"token": "e7c1334f53011a89d687c022b9706cbac752c50dda0dd2c8256c515e1ba2", "path": "/home/ftp/ftp/dev/wsr-public/data_output/WORLD_FATALITY_RATE.html", "filename": "../data_output/WORLD_FATALITY_RATE.html"}
# bob.static {"token": "8e1fc3f43fd3d171177998b992d81175eff70eb96f4c2d1151ae06823ed3", "path": "/home/ftp/ftp/dev/wsr-public/data_output/WORLD_RECOVERY_RATE.html", "filename": "../data_output/WORLD_RECOVERY_RATE.html"}

In [7]:
%run __init__.ipynb

Successfully connected to MongoDB


In [8]:
#--- Check if the data are already updated
filePath = f'{output_folder}103_WORLD.csv'
if path.exists(filePath):
    df_check = pd.read_csv(filePath, sep=';',low_memory=False)
    date_check = df_check[df_check['SCENARIO'] == yesterday.strftime('%d/%m/%Y')]
    if len(date_check) > 0:
        raise Exception(f'Data already updated ! DateTime : {now}')
        df.head(5)

In [9]:
df = pd.read_csv(output_folder + '103.csv',sep=";").drop('Unnamed: 0',axis=1)
filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
df = df[(df['ENTITY'].isin(filters)) & (df['SCENARIO'] == yesterday.strftime('%d/%m/%Y'))].reset_index(drop=True)
if len(df) == 0:
    raise Exception(f'Data not updated ! DateTime : {now}')
else:
    df.to_csv(output_folder + '103_WORLD.csv',sep=";")
df.head(5)

,ENTITY,LABEL,KPI,VALUE,VALUE_D-1,VAR,VARP,SCENARIO,DATE_ORDER,UNIT_VALUE,WORLDMAP,WORLDMAP_CS,CONTINENT_CS
0,Africa,Algeria,Confirmed,3517.0,3382.0,135.0,3.991721,27/04/2020,20200427,cases,Algeria,Algeria,Africa
1,Africa,Angola,Confirmed,27.0,26.0,1.0,3.846154,27/04/2020,20200427,cases,Angola,Angola,Africa
2,Africa,Benin,Confirmed,64.0,64.0,0.0,0.000000,27/04/2020,20200427,cases,Benin,Benin,Africa
3,Africa,Botswana,Confirmed,22.0,22.0,0.0,0.000000,27/04/2020,20200427,cases,Botswana,Botswana,Africa
4,Africa,Burkina Faso,Confirmed,635.0,632.0,3.0,0.474684,27/04/2020,20200427,cases,Burkina Faso,Burkina Faso,Africa


In [10]:
# df_list = pd.DataFrame()
# for i in range(0,232):
#     print(worldmap['features'][i]['properties']['name'])
#     list_country = [{'name': worldmap['features'][i]['properties']['name'],'continent':worldmap['features'][i]['properties']['continent'],'subregion': worldmap['features'][i]['properties']['subregion']}]
#     df_list = df_list.append(list_country)

# df_list.to_csv(output_folder + 'LIST_WORLD.csv',sep=";")
# df_list

In [11]:
#Var France
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map(df,geo,dfKey,geoKey,scale,colors,map_center,label,file_html):
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color="VALUE",
                               locations=dfKey, 
                               featureidkey=geoKey,
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=1, 
                               center=map_center,
                               hover_data=["VALUE","VAR"],
                               labels=label,
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(f"{output_folder}{file_html}")
    updateChartCss(f"{output_folder}{file_html}", f"{input_folder}map.css")
#     return fig

In [12]:
#Confirmed
kpi = 'Confirmed'
file_html = "WORLD_CONFIRMED.html"

df_map = df.copy()[(df['KPI'] == kpi)]
analysis(df_map,'VALUE')

scale = (0,500000)
colors = [
    [0.00, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.05, "#81B1BD"],
    [0.1, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "WORLDMAP_CS"
geoKey="properties.name"
to_rename = {'WORLDMAP_CS':'Country','VALUE': kpi, 'VAR': 'Var. in cases'}
create_map(df_map,worldmap,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 1.0, Max: 988197.0, Average: 16442.0, AvrMax: 0.02, Med: 712.0, MedMax: 0.0


In [13]:
#"Active cases"
kpi = "Active cases"
file_html = "WORLD_ACTIVE_CASES.html"

df_map = df.copy()[(df['KPI'] == kpi)]
analysis(df_map,'VALUE')

scale = (0,500000)
colors = [
    [0.00, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.05, "#81B1BD"],
    [0.1, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "WORLDMAP_CS"
geoKey="properties.name"
to_rename = {'WORLDMAP_CS':'Country','VALUE': kpi, 'VAR': 'Var. in cases'}
create_map(df_map,worldmap,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0.0, Max: 820514.0, Average: 10468.0, AvrMax: 0.01, Med: 433.0, MedMax: 0.0


In [14]:
#"Deaths"
kpi = "Deaths"
file_html = "WORLD_DEATHS.html"

df_map = df.copy()[(df['KPI'] == kpi)]
analysis(df_map,'VALUE')

scale = (0,50000)
colors = [
    [0.00, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.05, "#81B1BD"],
    [0.1, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "WORLDMAP_CS"
geoKey="properties.name"
to_rename = {'WORLDMAP_CS':'Country','VALUE': kpi, 'VAR': 'Var. in cases'}
create_map(df_map,worldmap,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0.0, Max: 56259.0, Average: 1141.0, AvrMax: 0.02, Med: 14.0, MedMax: 0.0


In [15]:
#"Recovered"
kpi = "Recovered"
file_html = "WORLD_RECOVERED.html"

df_map = df.copy()[(df['KPI'] == kpi)]
analysis(df_map,'VALUE')

scale = (0,100000)
colors = [
    [0.00, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.05, "#81B1BD"],
    [0.1, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "WORLDMAP_CS"
geoKey="properties.name"
to_rename = {'WORLDMAP_CS':'Country','VALUE': kpi, 'VAR': 'Var. in cases'}
create_map(df_map,worldmap,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0.0, Max: 120832.0, Average: 4832.0, AvrMax: 0.04, Med: 233.0, MedMax: 0.0


In [16]:
#"Fatality Rate"
kpi = "Fatality Rate"
file_html = "WORLD_FATALITY_RATE.html"

df_map = df.copy()[(df['KPI'] == kpi)]
df_map['VALUE'] = round(df_map['VALUE'],2)
df_map['VAR'] = round(df_map['VAR'],2)
analysis(df_map,'VALUE')

scale = (0,10)
colors = [
    [0.0, "#FFFFFF"],
    [0.25,"#B3D7D2"],
    [0.5, "#81B1BD"],
    [0.75, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "WORLDMAP_CS"
geoKey="properties.name"
to_rename = {'WORLDMAP_CS':'Country','VALUE': kpi, 'VAR': 'Var. in pts'}
create_map(df_map,worldmap,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0.0, Max: 23.08, Average: 4.0, AvrMax: 0.17, Med: 3.0, MedMax: 0.13


In [17]:
#"Recovery Rate"
kpi = "Recovery Rate"
file_html = "WORLD_RECOVERY_RATE.html"

df_map = df.copy()[(df['KPI'] == kpi)]
df_map['VALUE'] = round(df_map['VALUE'],2)
df_map['VAR'] = round(df_map['VAR'],2)
analysis(df_map,'VALUE')

scale = (0,100)
colors = [
    [0.0, "#FFFFFF"],
    [0.15,"#B3D7D2"],
    [0.25, "#81B1BD"],
    [0.5, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "WORLDMAP_CS"
geoKey="properties.name"
to_rename = {'WORLDMAP_CS':'Country','VALUE': kpi, 'VAR': 'Var. in pts'}
create_map(df_map,worldmap,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0.0, Max: 100.0, Average: 37.0, AvrMax: 0.37, Med: 31.0, MedMax: 0.31


In [18]:
#html
confirmed = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/2490a15907a84c6992746a301472670f93d69adfd300d878d6ca779bd1fa', 'KPI' : "Confirmed"}]
active_cases = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/a43c068d301b1bf55b676a083b4ee8fe8414f1802dfe0a2b70991b04a752', 'KPI' : "Active cases"}]
deaths = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/e69a785963750df22b746d3d35342a166a2832dc336e6e587049b0afe7f6', 'KPI' : "Deaths"}]
recovered = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/3bf81240bd15e2bb2560e1efdaeebd11252aead4a95563ab1d3a44d96505', 'KPI' : "Recovered"}]
fatality_rate = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/e7c1334f53011a89d687c022b9706cbac752c50dda0dd2c8256c515e1ba2', 'KPI' : "Fatality Rate"}]
recovery_rate = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/8e1fc3f43fd3d171177998b992d81175eff70eb96f4c2d1151ae06823ed3', 'KPI' : "Recovery Rate"}]


list_kpis = confirmed + active_cases + deaths + recovered + fatality_rate + recovery_rate
domain = pd.DataFrame(list_kpis)
domain.loc[:,'VALUE'] = """<iframe src='"""+ domain['URL'] +"""' frameborder='0' style='overflow:hidden;height:85vh;width:100%' height='100%' width='100%'></iframe> <style>.modebar{display: none!important;}</style>"""
if use_mongo:
    bob.mongo.save_df(domain,'111',db_app,True)
domain

Dataframe 111 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.0164334774017334 secnds ---


,URL,KPI,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Confirmed,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,Active cases,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Deaths,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Recovered,<iframe src='https://api.dev.cashstory.com/api...
4,https://api.dev.cashstory.com/api/v1/notebook/...,Fatality Rate,<iframe src='https://api.dev.cashstory.com/api...
5,https://api.dev.cashstory.com/api/v1/notebook/...,Recovery Rate,<iframe src='https://api.dev.cashstory.com/api...
